# 📊 Daily KPI Time-Series Analysis — EDA, Correlation, Forecasting (Prophet + LSTM)
Analyze and forecast business KPIs such as sales, users, and marketing performance using statistical and neural models.

In [ ]:
# STEP 1: Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from prophet import Prophet
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
sns.set(style='whitegrid')
pd.set_option('display.max_columns', None)


In [ ]:
# STEP 2: Load dataset
df = pd.read_csv('daily_kpi_data.csv', parse_dates=['date'])
print('Shape:', df.shape)
df.head()


In [ ]:
# STEP 3: Basic EDA
print('Date Range:', df['date'].min(), 'to', df['date'].max())
print('\nMissing values:\n', df.isnull().sum())

plt.figure(figsize=(10,5))
sns.lineplot(x='date', y='total_sales', data=df, label='Total Sales')
sns.lineplot(x='date', y='active_users', data=df, label='Active Users')
plt.title('Sales & Active Users Over Time')
plt.legend()
plt.show()


In [ ]:
# STEP 4: Correlation Heatmap
num_cols = ['total_sales','active_users','new_customers','marketing_spend','website_traffic','conversion_rate','customer_satisfaction']
plt.figure(figsize=(8,6))
sns.heatmap(df[num_cols].corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix Among KPIs')
plt.show()


In [ ]:
# STEP 5: Prophet Forecasting — Total Sales
sales_df = df[['date','total_sales']].rename(columns={'date':'ds','total_sales':'y'})
model = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=False)
model.fit(sales_df)
future = model.make_future_dataframe(periods=90)
forecast = model.predict(future)
fig1 = model.plot(forecast)
plt.title('Prophet Forecast — Total Sales')
plt.show()
fig2 = model.plot_components(forecast)
plt.show()


In [ ]:
# STEP 6: LSTM Forecasting
series = df.set_index('date')['total_sales'].asfreq('D').fillna(method='ffill')
data = series.values.reshape(-1,1)
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

def create_sequences(data, seq_len=30):
    X, y = [], []
    for i in range(len(data)-seq_len):
        X.append(data[i:i+seq_len])
        y.append(data[i+seq_len])
    return np.array(X), np.array(y)

SEQ_LEN = 30
X, y = create_sequences(data_scaled, SEQ_LEN)
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

model_lstm = Sequential([
    LSTM(64, input_shape=(SEQ_LEN,1), return_sequences=False),
    Dropout(0.2),
    Dense(1)
])
model_lstm.compile(optimizer='adam', loss='mse')
history = model_lstm.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.1, verbose=1)

preds = model_lstm.predict(X_test)
preds_inv = scaler.inverse_transform(preds)
y_test_inv = scaler.inverse_transform(y_test)
plt.figure(figsize=(10,5))
plt.plot(range(len(y_test_inv)), y_test_inv, label='Actual')
plt.plot(range(len(preds_inv)), preds_inv, label='Predicted')
plt.title('LSTM Forecasting — Total Sales (Test Set)')
plt.legend()
plt.show()


## ✅ Insights & Next Steps
- Prophet reveals seasonality trends and recurring patterns in sales and traffic.
- LSTM captures complex dependencies — useful for multi-KPI forecasting.
- Next: Add marketing spend, users, or satisfaction as regressors in Prophet.
- Extend to multi-variate deep models (VAR, Seq2Seq, or Transformers).
